In [138]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline
import os
import math

############################################################################################
def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def calc_slope(x1, y1, x2, y2):
    return (y2 - y1) / (x2 - x1)

def calc_slope_line(line):
    return calc_slope(line[0],line[1],line[2],line[3])

def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    right_line = [0] * 4
    right_total = 0
    left_line = [0] * 4
    left_total = 0
    for line in lines:
        for x1,y1,x2,y2 in line:
            slope = calc_slope(x1,y1,x2,y2)
            if(slope > 0):
                left_line[0] += x1
                left_line[1] += y1
                left_line[2] += x2
                left_line[3] += y2
                left_total += 1
            elif(slope < 0):
                right_line[0] += x1
                right_line[1] += y1
                right_line[2] += x2
                right_line[3] += y2
                right_total += 1

    right_average_line = average_line(right_line, right_total)
    left_average_line = average_line(left_line, left_total)

    imshape = img.shape
    
    right_average_line = extrapolated_line(right_average_line, imshape)
    left_average_line = extrapolated_line(left_average_line, imshape)
    
    averaged_lines = [right_average_line, left_average_line]
    
    for line in averaged_lines:
            cv2.line(img, (int(line[0]), int(line[1])), (int(line[2]), int(line[3])), color, thickness)

def extrapolated_line(line, image_shape):
    y_boundary = 320
    
    slope = calc_slope_line(line)
    b = -1*slope*line[0] + line[1]
    x_min = (image_shape[0] - b) / slope
    x_max = (y_boundary - b) / slope
    
    line[0] = x_min
    line[1] = image_shape[0]
    line[2] = x_max
    line[3] = y_boundary
    
    return line

def average_line(line, total):
    for i in range(4):
        line[i] = line[i] / total
    return line
        

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((*img.shape, 3), dtype=np.uint8)
    draw_lines(line_img, lines, thickness=9)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)
############################################################################################

def allImagesInDirectoryPipeline(directory):
    list_images = os.listdir(directory)
    for image in list_images:
        pipeline(directory, image)


def pipeline(directory, file):
    initial_image = (mpimg.imread(directory + file)).astype('uint8')
    
    #grayscale image
    gray_image = grayscale(initial_image)
    
    #blur image
    kernel_size = 5
    blur_gray_image = gaussian_blur(gray_image, kernel_size)
    
    #Canny image
    low_threshold = 10
    high_threshold = 150
    edges_image = canny(blur_gray_image, low_threshold, high_threshold)
    
    #take the image shape
    imshape = edges_image.shape
    
    #mask
    vertices = np.array([[(100,imshape[0]),(450, 290), (490, 290), (imshape[1],imshape[0])]], dtype=np.int32)
    region_image = region_of_interest(edges_image, vertices)
    #plt.imshow(region_image)
    
    #hough
    rho = 1 # distance resolution in pixels of the Hough grid
    theta = 1.25*np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 45    # minimum number of votes (intersections in Hough grid cell)
    min_line_length = 40 #minimum number of pixels making up a line
    max_line_gap = 20   # maximum gap in pixels between connectable line segments
    line_image = np.copy(initial_image)*0 # creating a blank to draw lines on
    hough_image = hough_lines(region_image, rho, theta, threshold, min_line_length, max_line_gap)

    #weighted image
    weighted_image = weighted_img(hough_image, initial_image)
        
    #plt.imshow(weighted_image)
    mpimg.imsave(directory + "new-" + file , weighted_image)
    return

#pipeline("test_images/", "whiteCarLaneSwitch.jpg")
allImagesInDirectoryPipeline("test_images/")

In [ ]:
import os
os.list